In [10]:
# figures for harmony 2023
import collections

import compress_pickle
import copy
import editdistance
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
FIGURE_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/AMAS_suppl/figure_files'
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubshin/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")

from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

In [25]:
def printReaction(one_r, name=False, ret=False):
  """
  Print out a reaction using 
  a libsbml.Reaction instance. 
  
  Parameters
  ----------
  one_r: libsbml.Reaction
  
  name: bool
      If True, prints out using display names
  
  ret: bool
      If True, returns a string
  """
  if name: 
    reactants = [model.getSpecies(val.species).name \
                 for val in one_r.getListOfReactants()]
    products = [model.getSpecies(val.species).name \
                for val in one_r.getListOfProducts()]
  else:
    reactants = [val.species \
                 for val in one_r.getListOfReactants()]
    products = [val.species \
                for val in one_r.getListOfProducts()]
  reacts_str = ' + '.join(reactants)
  prods_str = ' + '.join(products)
  result = "%s: %s" % (one_r.getId(), reacts_str + ' => ' + prods_str)
  if ret:
    return result
  else:
    print(result)

In [11]:
BIOMD_190_PATH = os.path.join(cn.TEST_DIR, 'BIOMD0000000190.xml')
recom = recommender.Recommender(libsbml_fpath=BIOMD_190_PATH)
model = recom.sbml_document.getModel()

In [12]:
recom.reactions.exist_annotation

{'ODC': ['RHEA:22964'],
 'SAMdc': ['RHEA:15981'],
 'SSAT_for_S': ['RHEA:11116', 'RHEA:33099', 'RHEA:28270', 'RHEA:28150'],
 'SSAT_for_D': ['RHEA:11116', 'RHEA:33099', 'RHEA:28270', 'RHEA:28150'],
 'PAO_for_aD': ['RHEA:25800', 'RHEA:16133'],
 'PAO_for_aS': ['RHEA:25800', 'RHEA:16133'],
 'SpdS': ['RHEA:12721'],
 'SpmS': ['RHEA:19973'],
 'MAT': ['RHEA:21080']}

In [28]:
recom2 = recommender.Recommender(libsbml_fpath=os.path.join(BIGG_DIR, 'iSB619.xml'))
print(recom2.reactions.exist_annotation['R_ORNDC'])

model2 = recom2.sbml_document.getModel()
one_r = model2.getReaction('R_ORNDC')
printReaction(one_r)

['RHEA:22964']
R_ORNDC: M_h_c + M_orn_c => M_co2_c + M_ptrc_c


In [17]:
recom190 = recommender.Recommender(libsbml_fpath=BIOMD_190_PATH)
reahs190 = recom190.reactions.exist_annotation

comms = []

biggs = [val for val in os.listdir(BIGG_DIR) if val[-4:]=='.xml']
for idx, one_bigg in enumerate(biggs):
  if idx % 10 == 0:
    print("We're at: %d" % idx)
  one_recom = recommender.Recommender(libsbml_fpath=os.path.join(BIGG_DIR, one_bigg))
  one_rheas = one_recom.reactions.exist_annotation
  for one_k in one_rheas.keys():
    if any(set(one_rheas[one_k]).intersection(reahs190['ODC'])):
      comms.append((one_bigg, one_k, len(one_rheas)))
      break

We're at: 0
We're at: 10
We're at: 20
We're at: 30
We're at: 40
We're at: 50
We're at: 60
We're at: 70
We're at: 80
We're at: 90
We're at: 100


In [31]:
print(model.getReaction('ODC').getAnnotationString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqmodel="http://biomodels.net/model-qualifiers/" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000062">
      <bqbiol:isVersionOf>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/ec-code/4.1.1.17"/>
          <rdf:li rdf:resource="http://identifiers.org/kegg.reaction/R00670"/>
          <rdf:li rdf:resource="http://identifiers.org/reactome/REACT_1211.3"/>
        </rdf:Bag>
      </bqbiol:isVersionOf>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [22]:
for one_r in model.getListOfReactions():
  printReaction(one_r, name=False)

ODC: ORN => P
SAMdc: SAM => A
SSAT_for_S: S + AcCoA => aS + CoA
SSAT_for_D: D + AcCoA => aD + CoA
PAO_for_aD: aD => P
PAO_for_aS: aS => D
SpdS: A + P => D
SpmS: A + D => S
MAT: Met => SAM
VCoA: AcCoA => CoA
VacCoA: CoA => AcCoA
P_efflux: P => 
aD_efflux: aD => 


In [7]:
for one_r in model.getListOfReactions():
  reactants = [val.species \
               for val in one_r.getListOfReactants()]
  products = [val.species \
              for val in one_r.getListOfProducts()]
  reacts_str = ' + '.join(reactants)
  prods_str = ' + '.join(products)
  print("%s: %s" % (one_r.getId(), reacts_str + ' => ' + prods_str))

ODC: ORN => P
SAMdc: SAM => A
SSAT_for_S: S + AcCoA => aS + CoA
SSAT_for_D: D + AcCoA => aD + CoA
PAO_for_aD: aD => P
PAO_for_aS: aS => D
SpdS: A + P => D
SpmS: A + D => S
MAT: Met => SAM
VCoA: AcCoA => CoA
VacCoA: CoA => AcCoA
P_efflux: P => 
aD_efflux: aD => 


In [6]:
for one_r in model.getListOfReactions():
  reactants = [model.getSpecies(val.species).name \
               for val in one_r.getListOfReactants()]
  products = [model.getSpecies(val.species).name \
              for val in one_r.getListOfProducts()]
  reacts_str = ' + '.join(reactants)
  prods_str = ' + '.join(products)
  print("%s: %s" % (one_r.getId(), reacts_str + ' => ' + prods_str))

ODC: L-Ornithine => Putrescine
SAMdc: S-adenosyl-L-methionine => S-adenosylmethioninamine
SSAT_for_S: Spermine + Acetyl-CoA => N1-Acetylspermine + CoA
SSAT_for_D: Spermidine + Acetyl-CoA => N1-Acetylspermidine + CoA
PAO_for_aD: N1-Acetylspermidine => Putrescine
PAO_for_aS: N1-Acetylspermine => Spermidine
SpdS: S-adenosylmethioninamine + Putrescine => Spermidine
SpmS: S-adenosylmethioninamine + Spermidine => Spermine
MAT: Methionine => S-adenosyl-L-methionine
VCoA: Acetyl-CoA => CoA
VacCoA: CoA => Acetyl-CoA
P_efflux: Putrescine => 
aD_efflux: N1-Acetylspermidine => 


In [34]:
recom.recommendReaction(['ODC'])

                                        ODC (credibility score: 0.815)                                       
+----+--------------+---------------+-----------------------------------------------------------------------+
|    | annotation   |   match score | label                                                                 |
+====+==============+===============+=======================================================================+
|  1 | RHEA:28827   |         1.000 | L-ornithine(out) + putrescine(in) = L-ornithine(in) + putrescine(out) |
+----+--------------+---------------+-----------------------------------------------------------------------+
|  2 | RHEA:22964   |         0.500 | ornithine decarboxylase activity                                      |
+----+--------------+---------------+-----------------------------------------------------------------------+
|  3 | RHEA:59048   |         0.500 | D-ornithine + H(+) = CO2 + putrescine                                 |
+----+----

In [39]:
df = recom.just_displayed['ODC']
pd.set_option('display.max_colwidth', None)
df.loc[:, ['annotation', 'label']]

,annotation,label
ODC (cred. 0.815),,
1,RHEA:28827,L-ornithine(out) + putrescine(in) = L-ornithine(in) + putrescine(out)
2,RHEA:22964,ornithine decarboxylase activity
3,RHEA:59048,D-ornithine + H(+) = CO2 + putrescine


In [40]:
recom2.recommendReaction(['R_ORNDC'])

                      R_ORNDC (credibility score: 0.937)                     
+----+--------------+---------------+---------------------------------------+
|    | annotation   |   match score | label                                 |
+====+==============+===============+=======================================+
|  1 | RHEA:22964   |         1.000 | ornithine decarboxylase activity      |
+----+--------------+---------------+---------------------------------------+
|  2 | RHEA:59048   |         1.000 | D-ornithine + H(+) = CO2 + putrescine |
+----+--------------+---------------+---------------------------------------+



In [45]:
df = recom2.just_displayed['R_ORNDC']
pd.set_option('display.max_colwidth', None)
df.index.name = None
df

,annotation,match score,label
1,RHEA:22964,1.0,ornithine decarboxylase activity
2,RHEA:59048,1.0,D-ornithine + H(+) = CO2 + putrescine


In [48]:
df.to_csv('/Users/woosubshin/Desktop/AutomateAnnotation/Conference/harmony2023/bigg_r_orndc_recom.csv')

In [60]:
# annotation statistics
biomd_all_specs = 0
biomd_anot_specs = 0
biomd_all_reacs = 0
biomd_anot_reacs = 0

biomds = [val for val in os.listdir(BIOMODEL_DIR) if val[-4:]=='.xml']
for idx, one_biomd in enumerate(biomds):
  if idx % 200 == 0:
    print("We're at", idx)
  reader = libsbml.SBMLReader()
  document = reader.readSBML(os.path.join(BIOMODEL_DIR, one_biomd))
  model = document.getModel()
  biomd_all_specs += model.getNumSpecies()
  for one_spec in model.getListOfSpecies():
    annotation = one_spec.getAnnotationString()
    if 'bqbiol:is' in annotation or 'bqbiol:isVersionOf' in annotation:
      biomd_anot_specs += 1

    
  biomd_all_reacs += model.getNumReactions()
  for one_reac in model.getListOfReactions():
    annotation = one_reac.getAnnotationString()
    if 'bqbiol:is' in annotation or 'bqbiol:isVersionOf' in annotation:
      biomd_anot_reacs += 1

We're at 0
We're at 200
We're at 400
We're at 600
We're at 800
Species: 15252 / 23255
Reactions: 14875 / 30094


In [61]:
print('Species: %d / %d = %.02f' % (biomd_anot_specs,
                                    biomd_all_specs,
                                    biomd_anot_specs/biomd_all_specs))
print('Reactions: %d / %d = %.02f' % (biomd_anot_reacs,
                                      biomd_all_reacs,
                                      biomd_anot_reacs/biomd_all_reacs))

Species: 15252 / 23255 = 0.66
Reactions: 14875 / 30094 = 0.49


In [59]:
biomd_anot_specs

15252

In [ ]:
# Slide2: bar plots of annotations